# Settings

In [ ]:
root = '' # change as needed

data_root = root + 'data/'
data_file = 'hcp1003_REST1_LR_dbs80.mat'
sc_file = 'SC_dbs80HARDIFULL.mat'
save_path = root + 'results3/G/'
!mkdir -p {save_path}

# Number of simulations
n_simulated_subjects = 20

# G range
G_min, G_max, G_step = 0, 8, 0.1

# Setup

In [ ]:
import numpy as np
import mat73
import scipy.io
import matplotlib.pyplot as plt

In [ ]:
# Whole brain model
import WholeBrain.Models.supHopf as Hopf
Hopf.initialValueX = Hopf.initialValueY = 0.1

# Integrator
import WholeBrain.Integrators.EulerMaruyama as scheme
scheme.neuronalModel = Hopf
import WholeBrain.Integrators.Integrator as integrator
integrator.integrationScheme = scheme
integrator.neuronalModel = Hopf
integrator.verbose = False

# Simulator
import WholeBrain.Utils.simulate_SimOnly as simulateBOLD
simulateBOLD.warmUp = True
simulateBOLD.warmUpFactor = 606./2000.
simulateBOLD.integrator = integrator

# Observable
import WholeBrain.Observables.phFCD as phFCD

# Method
import WholeBrain.Optimizers.ParmSweep as ParmSweep
ParmSweep.simulateBOLD = simulateBOLD
ParmSweep.integrator = integrator
ParmSweep.verbose = True

# Filters
import WholeBrain.Observables.filteredPowerSpectralDensity as filtPowSpectr
import WholeBrain.Observables.BOLDFilters as BOLDFilters
BOLDFilters.flp = 0.008      # lowpass frequency of filter
BOLDFilters.fhi = 0.08       # highpass
BOLDFilters.TR = 2.

In [ ]:
# Read time series
data = mat73.loadmat(data_root+data_file)
timeseries = np.array([i['dbs80ts'] for i in data['subject']])[:n_simulated_subjects]
all_fMRI = {s: d for s,d in enumerate(timeseries)}
S, N, Tmax = timeseries.shape

# Get SC
mat0 = scipy.io.loadmat(data_root+sc_file)['SC_dbs80FULL']
SCnorm = 0.2 * mat0 / mat0.max()
Hopf.setParms({'SC': SCnorm})
Hopf.couplingOp.setParms(SCnorm)

# Set observables
selectedObservable = 'phFCD'
distanceSettings = {'phFCD': (phFCD, True)}

# Set times
simulateBOLD.TR = BOLDFilters.TR   # Recording interval: 1 sample every X seconds
simulateBOLD.dt = 0.1 * simulateBOLD.TR / 2.
simulateBOLD.Tmax = Tmax  # This is the length, in seconds
simulateBOLD.dtt = simulateBOLD.TR  # We are not using milliseconds
simulateBOLD.t_min = 10 * simulateBOLD.TR
simulateBOLD.Tmaxneuronal = (Tmax-1) * simulateBOLD.TR + 30
integrator.ds = simulateBOLD.TR  # record every TR millisecond

# Frequency vector

In [ ]:
# Find frequencies
f_diff = filtPowSpectr.filtPowSpetraMultipleSubjects(timeseries, TR=BOLDFilters.TR)
f_diff[np.where(f_diff == 0)] = np.mean(f_diff[np.where(f_diff != 0)])

# Set Hopf parameters
base_a_value = -0.02
Hopf.setParms({'a': base_a_value})
Hopf.setParms({'omega': 2 * np.pi * f_diff})

In [ ]:
w = 2*np.pi*f_diff
np.save(save_path+f'{n_simulated_subjects}_w.npy', w)

In [ ]:
w

# Coupling factor G

## Simulate

In [ ]:
# Single subject pipeline
def preprocessingPipeline(all_fMRI,
                          distanceSettings,  # This is a dictionary of {name: (distance module, apply filters bool)}
                          G):
    balancedParms = [{'G': g} for g in G]
    fitting = ParmSweep.distanceForAll_Parms(all_fMRI, G, balancedParms, NumSimSubjects=n_simulated_subjects,
                                            observablesToUse=distanceSettings,
                                            parmLabel='G',
                                            outFilePath=save_path)
    optimal = {}
    for sd in distanceSettings:
        optim = distanceSettings[sd][0].findMinMax(fitting[sd])
        optimal[sd] = (optim[0], optim[1], balancedParms[optim[1]])
    return optimal

plt.rcParams.update({'font.size': 22})
G = np.arange(G_min, G_max + G_step, G_step)
optimal = preprocessingPipeline(all_fMRI, distanceSettings, G)

## Read generated files and find the optimal value

In [ ]:
import glob

sim_files = save_path + '/fitting_G{}.mat'
emp_file = save_path + '/fNeuro_emp.mat'
files = sorted(glob.glob(sim_files.format("*")))

emp_values = mat73.loadmat(emp_file)['phFCD']
measure_function = distanceSettings['phFCD'][0]

x = np.zeros(len(files), dtype=np.float64)
y = np.zeros(len(files), dtype=np.float64)
for i,(f,g) in enumerate(zip(files,G)):
	print(g)
	file_content = mat73.loadmat(f)
	sim_values = file_content['phFCD']
	distance = measure_function.distance(emp_values, sim_values)
	x[i] = file_content['G'].squeeze()
	y[i] = distance

In [ ]:
from scipy.signal import find_peaks_cwt
import matplotlib.pyplot as plt
import numpy as np

# Fit polynomial curve
z = np.polynomial.polynomial.Polynomial.fit(x, y, 8)

# Find peak positions
peak_id = find_peaks_cwt(-z(x), np.arange(1, 100))

# Create figure and set size
plt.figure(figsize=(8, 6))

# Plot data and polynomial fit
plt.plot(x, y, label='Parameter sweep', linewidth=2, color='steelblue')
plt.plot(x, z(x), label='Polynomial fit (degree 8)', color='darkred', linewidth=2, linestyle='--')

# Mark the peak and annotate
plt.plot(x[peak_id], z(x)[peak_id], marker='o', markersize=8, color='black')
plt.text(
    x[peak_id], z(x)[peak_id] + 0.05, f'$G = {x[peak_id][0]:.1f}$',
    fontsize=11, ha='center', va='top', fontweight='bold'
)

# Add axis labels, title, and grid
plt.xlabel('Coupling factor G', fontsize=14)
plt.ylabel('Kolmogorov-Smirnov distance', fontsize=14)

# Adjust tick label size
plt.tick_params(axis='both', which='major', labelsize=10)

# Add grid and legend
plt.grid(True, which='both', linestyle='--', linewidth=0.7, alpha=0.7)
plt.legend(fontsize=12, loc='best')

# Improve layout
plt.tight_layout()

# Display and save the plot
plt.savefig(save_path+f'{n_simulated_subjects}_g.png', dpi=800)